In [186]:
import pandas as pd
from scipy import stats
from scipy.stats import mode
from sklearn.preprocessing import LabelEncoder

train = pd.read_csv('/home/prashant/Desktop/competation/bigmartsales.csv')
test = pd.read_csv('/home/prashant/Desktop/competation/bigmartsales2.csv')

In [187]:
train['source']='train'
test['source']='test'
salesdata = pd.concat([train, test],ignore_index=True)

In [188]:
salesdata.isnull().sum()

Item_Fat_Content                0
Item_Identifier                 0
Item_MRP                        0
Item_Outlet_Sales            5681
Item_Type                       0
Item_Visibility                 0
Item_Weight                  2439
Outlet_Establishment_Year       0
Outlet_Identifier               0
Outlet_Location_Type            0
Outlet_Size                  4016
Outlet_Type                     0
source                          0
dtype: int64

In [189]:
#Converting catagorical values of 'Outlet_Size' to Integer Values
cat_dict ={}
unique_outlet_size_val = salesdata.Outlet_Size.dropna().unique()
copy_outlet_size = salesdata.Outlet_Size.dropna()
count =0
for val in unique_outlet_size_val:
    cat_dict[val] =int(count)
    count =count+1
salesdata['Outlet_Size'] =copy_outlet_size.apply(lambda x :cat_dict[x])


In [190]:
#calculating mode of the column Outlet_Size and mapping each mode to its Outlet_Type
mode_fn =salesdata.pivot_table(columns ='Outlet_Type',values = 'Outlet_Size',aggfunc = (lambda x : (mode(x).mode[0]))).astype(int) 

miss_val = salesdata.Outlet_Size.isnull()
#print(miss_val)
#print(salesdata.loc[miss_val,'Outlet_Type'])
salesdata.loc[miss_val,'Outlet_Size'] = salesdata.loc[miss_val,'Outlet_Type'].apply(lambda x : mode_fn[x])
print('missing values in Outlet_size %s'%salesdata.isnull().sum())

missing values in Outlet_size Item_Fat_Content                0
Item_Identifier                 0
Item_MRP                        0
Item_Outlet_Sales            5681
Item_Type                       0
Item_Visibility                 0
Item_Weight                  2439
Outlet_Establishment_Year       0
Outlet_Identifier               0
Outlet_Location_Type            0
Outlet_Size                     0
Outlet_Type                     0
source                          0
dtype: int64


In [191]:
#Filling missing data for 'Item_Weight' feild
avg_item_weight = salesdata.pivot_table(index = 'Item_Identifier',values ='Item_Weight')
avg_item_weight
miss_weight = salesdata.Item_Weight.isnull()
salesdata.loc[miss_weight , 'Item_Weight'] = salesdata.loc[miss_weight,'Item_Identifier'].apply(lambda x : avg_item_weight[x])

salesdata.isnull().sum()



Item_Fat_Content                0
Item_Identifier                 0
Item_MRP                        0
Item_Outlet_Sales            5681
Item_Type                       0
Item_Visibility                 0
Item_Weight                     0
Outlet_Establishment_Year       0
Outlet_Identifier               0
Outlet_Location_Type            0
Outlet_Size                     0
Outlet_Type                     0
source                          0
dtype: int64

In [192]:
salesdata.head()

,Item_Fat_Content,Item_Identifier,Item_MRP,Item_Outlet_Sales,Item_Type,Item_Visibility,Item_Weight,Outlet_Establishment_Year,Outlet_Identifier,Outlet_Location_Type,Outlet_Size,Outlet_Type,source
0,Low Fat,FDA15,249.8092,3735.1380,Dairy,0.016047,9.30,1999,OUT049,Tier 1,0.0,Supermarket Type1,train
1,Regular,DRC01,48.2692,443.4228,Soft Drinks,0.019278,5.92,2009,OUT018,Tier 3,0.0,Supermarket Type2,train
2,Low Fat,FDN15,141.6180,2097.2700,Meat,0.016760,17.50,1999,OUT049,Tier 1,0.0,Supermarket Type1,train
3,Regular,FDX07,182.0950,732.3800,Fruits and Vegetables,0.000000,19.20,1998,OUT010,Tier 3,2.0,Grocery Store,train
4,Low Fat,NCD19,53.8614,994.7052,Household,0.000000,8.93,1987,OUT013,Tier 3,1.0,Supermarket Type1,train


In [193]:
#filling Item_fat_content effectively

salesdata.Item_Fat_Content = salesdata.Item_Fat_Content.replace({ 'LF':'Low Fat','reg':'Regular','low fat':'Low Fat'})
salesdata.Item_Fat_Content.value_counts()

Low Fat    9185
Regular    5019
Name: Item_Fat_Content, dtype: int64

In [194]:
#FEATURE ENGINEERING ---- creating  a new column
salesdata['total_years'] = 2017 - salesdata['Outlet_Establishment_Year']


In [195]:
#filling 0 with mean item visibility
item_vis = salesdata.pivot_table(index = 'Item_Identifier' , values= 'Item_Visibility')
salesdata.Item_Visibility=salesdata.loc[(salesdata['Item_Visibility'] ==0),'Item_Identifier'].apply(lambda x : item_vis[x])


In [196]:
print(len(salesdata['Item_Type'].unique()))
salesdata['Item_Type'].value_counts()

16


Fruits and Vegetables    2013
Snack Foods              1989
Household                1548
Frozen Foods             1426
Dairy                    1136
Baking Goods             1086
Canned                   1084
Health and Hygiene        858
Meat                      736
Soft Drinks               726
Breads                    416
Hard Drinks               362
Others                    280
Starchy Foods             269
Breakfast                 186
Seafood                    89
Name: Item_Type, dtype: int64

In [197]:
salesdata.head()


,Item_Fat_Content,Item_Identifier,Item_MRP,Item_Outlet_Sales,Item_Type,Item_Visibility,Item_Weight,Outlet_Establishment_Year,Outlet_Identifier,Outlet_Location_Type,Outlet_Size,Outlet_Type,source,total_years
0,Low Fat,FDA15,249.8092,3735.1380,Dairy,NaN,9.30,1999,OUT049,Tier 1,0.0,Supermarket Type1,train,18
1,Regular,DRC01,48.2692,443.4228,Soft Drinks,NaN,5.92,2009,OUT018,Tier 3,0.0,Supermarket Type2,train,8
2,Low Fat,FDN15,141.6180,2097.2700,Meat,NaN,17.50,1999,OUT049,Tier 1,0.0,Supermarket Type1,train,18
3,Regular,FDX07,182.0950,732.3800,Fruits and Vegetables,0.017834,19.20,1998,OUT010,Tier 3,2.0,Grocery Store,train,19
4,Low Fat,NCD19,53.8614,994.7052,Household,0.009780,8.93,1987,OUT013,Tier 3,1.0,Supermarket Type1,train,30


In [198]:
rep_food ={'FD':'Consumable_Food','DR':'Drinking_Food','NC':'Non_Consumable'}
salesdata['represent_food'] = salesdata.Item_Identifier.apply(lambda x : x[0:2]).map(lambda x: rep_food[x])
salesdata.represent_food.value_counts()

Consumable_Food    10201
Non_Consumable      2686
Drinking_Food       1317
Name: represent_food, dtype: int64

In [199]:
'''le = LabelEncoder()
salesdata['Outlet'] = le.fit_transform(salesdata['Outlet_Identifier'])
all_variable= list(salesdata.columns.values)
cat_data =[] 
index=0
for i in salesdata.dtypes:
    
    if i == 'object' and all_variable[index] != 'Outlet_Identifier' and all_variable[index] != 'Item_Identifier'   :
        cat_data.append(all_variable[index])
    index = index+1
print(cat_data)
index=0
le = LabelEncoder()
for i in cat_data :
    salesdata[i]= le.fit_transform(salesdata[i])
   '''

le = LabelEncoder()
#New variable for outlet
salesdata['Outlet'] = le.fit_transform(salesdata['Outlet_Identifier'])
var_mod = ['Item_Fat_Content','Outlet_Location_Type','Outlet_Size','represent_food','Outlet_Type','Outlet']
le = LabelEncoder()
for i in var_mod:
    salesdata[i] = le.fit_transform(salesdata[i])
    
#One Hot Coding:
salesdata = pd.get_dummies(salesdata, columns=['Item_Fat_Content','Outlet_Location_Type','Outlet_Size','Outlet_Type',
                              'represent_food','Outlet'])

In [200]:

salesdata.dtypes.count

<bound method Series.count of Item_Identifier               object
Item_MRP                     float64
Item_Outlet_Sales            float64
Item_Type                     object
Item_Visibility              float64
Item_Weight                  float64
Outlet_Establishment_Year      int64
Outlet_Identifier             object
source                        object
total_years                    int64
Item_Fat_Content_0           float64
Item_Fat_Content_1           float64
Outlet_Location_Type_0       float64
Outlet_Location_Type_1       float64
Outlet_Location_Type_2       float64
Outlet_Size_0                float64
Outlet_Size_1                float64
Outlet_Size_2                float64
Outlet_Type_0                float64
Outlet_Type_1                float64
Outlet_Type_2                float64
Outlet_Type_3                float64
represent_food_0             float64
represent_food_1             float64
represent_food_2             float64
Outlet_0                     float64
Outlet_1

In [201]:
train = salesdata.loc[salesdata['source']=="train"]
test = salesdata.loc[salesdata['source']=="test"]

test.drop(['Item_Outlet_Sales','source'],axis=1,inplace=True)
train.drop(['source'],axis=1,inplace=True)

train.to_csv("train_modified.csv",index=False)
test.to_csv("test_modified.csv",index=False)

/home/prashant/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/prashant/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [202]:
mean_sales = train['Item_Outlet_Sales'].mean()
print(mean_sales)

2181.2889135750365


In [203]:
base1 = test[['Item_Identifier','Outlet_Identifier']]
base1['Item_Outlet_Sales'] = mean_sales

base1.to_csv("/home/prashant/Desktop/competation/sol.csv",index=False)

/home/prashant/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
